In [14]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Importation et nettoyage des données

In [15]:
df = pd.read_csv(r'C:\Users\pasqu\Documents\GitHub\deepLearning\deep_learning\data\diamonds.csv')
df = df.drop(df.columns[0], axis=1)
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [16]:
cut_map = {
    'Fair': 0,
    'Good': 1,
    'Very Good': 2,
    'Premium': 3,
    'Ideal': 4
}

color_map = {
    'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6
}

clarity_map = {
    'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7
}


df['cut'] = df['cut'].map(cut_map)
df['color'] = df['color'].map(color_map)
df['clarity'] = df['clarity'].map(clarity_map)


df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,4,5,1,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,5,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,5,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,1,3,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,0,1,63.3,58.0,335,4.34,4.35,2.75


In [17]:
feature_cols = ['color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
target_cols = ['price', 'carat']
X = df[feature_cols]
y = df[target_cols]

X.head()

,color,clarity,depth,table,x,y,z
0,5,1,61.5,55.0,3.95,3.98,2.43
1,5,2,59.8,61.0,3.89,3.84,2.31
2,5,4,56.9,65.0,4.05,4.07,2.31
3,1,3,62.4,58.0,4.20,4.23,2.63
4,0,1,63.3,58.0,4.34,4.35,2.75


In [18]:
y.head()

,price,carat
0,326,0.23
1,326,0.21
2,327,0.23
3,334,0.29
4,335,0.31


In [32]:
X_np = X.values.astype(np.float32)
y_np = y.values.astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
y_train = scaler_y.fit_transform(y_train)
X_test = scaler_X.transform(X_test)
y_test = scaler_y.transform(y_test)

Modèle


In [34]:
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

input_dimension = X_train.shape[1]
output_dimension = y_train.shape[1]


In [50]:
p = 0.5
layer1 = 256
layer2  = 128
layer3 = 64
activation = 0.1

modele = nn.Sequential(
    nn.Linear(input_dimension, layer1),
    nn.BatchNorm1d(layer1),
    nn.LeakyReLU(activation),
    nn.Dropout(p),

    nn.Linear(layer1, layer2),
    nn.BatchNorm1d(layer2),
    nn.LeakyReLU(activation),
    nn.Dropout(p),

    nn.Linear(layer2, layer3),
    nn.BatchNorm1d(layer3),
    nn.LeakyReLU(activation),

    nn.Linear(layer3, output_dimension)
)

criterion = (nn.MSELoss())
optimizer = optim.Adam(modele.parameters(), lr=0.005)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=20, factor=0.5)
summary(modele, input_size=input_dimension, output_size=output_dimension)

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            2,048
├─BatchNorm1d: 1-2                       512
├─LeakyReLU: 1-3                         --
├─Dropout: 1-4                           --
├─Linear: 1-5                            32,896
├─BatchNorm1d: 1-6                       256
├─LeakyReLU: 1-7                         --
├─Dropout: 1-8                           --
├─Linear: 1-9                            8,256
├─BatchNorm1d: 1-10                      128
├─LeakyReLU: 1-11                        --
├─Linear: 1-12                           130
Total params: 44,226
Trainable params: 44,226
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            2,048
├─BatchNorm1d: 1-2                       512
├─LeakyReLU: 1-3                         --
├─Dropout: 1-4                           --
├─Linear: 1-5                            32,896
├─BatchNorm1d: 1-6                       256
├─LeakyReLU: 1-7                         --
├─Dropout: 1-8                           --
├─Linear: 1-9                            8,256
├─BatchNorm1d: 1-10                      128
├─LeakyReLU: 1-11                        --
├─Linear: 1-12                           130
Total params: 44,226
Trainable params: 44,226
Non-trainable params: 0

In [51]:
num_epochs = 300
best_loss = float('inf')


print("Début de l'entraînement...")

for epoch in range(num_epochs):
    modele.train() # Mode entraînement

    # Forward & Backward
    outputs = modele(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Évaluation sur le Test set (pour le monitoring)
    modele.eval() # Mode évaluation (désactive Dropout/BatchNorm updates)
    with torch.no_grad():
        test_outputs = modele(X_test_tensor)
        test_loss = criterion(test_outputs, y_test_tensor)

    # Update du scheduler
    scheduler.step(test_loss)

    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}] | Train Loss: {loss.item():.4f} | Test Loss: {test_loss.item():.4f}')

print("\nEntraînement terminé.")

Début de l'entraînement...
Epoch [50/300] | Train Loss: 0.0599 | Test Loss: 0.0251
Epoch [100/300] | Train Loss: 0.0482 | Test Loss: 0.0250
Epoch [150/300] | Train Loss: 0.0456 | Test Loss: 0.0231
Epoch [200/300] | Train Loss: 0.0468 | Test Loss: 0.0230
Epoch [250/300] | Train Loss: 0.0439 | Test Loss: 0.0227
Epoch [300/300] | Train Loss: 0.0436 | Test Loss: 0.0223

Entraînement terminé.


In [52]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


def get_metrics(model, X_tensor, y_true_scaled, scaler_y, dataset_name):
    """
    Calcule les métriques pour un dataset donné (Train ou Test)
    """
    model.eval()
    with torch.no_grad():
        preds_scaled = model(X_tensor).numpy()

    # Retour aux valeurs réelles
    preds_real = scaler_y.inverse_transform(preds_scaled)
    y_true_real = scaler_y.inverse_transform(y_true_scaled)

    results = []
    targets = ['Price ($)', 'Carat (ct)']

    for i, target_name in enumerate(targets):
        y_true = y_true_real[:, i]
        y_pred = preds_real[:, i]

        results.append({
            'Dataset': dataset_name,
            'Cible': target_name,
            'RMSE': round(np.sqrt(mean_squared_error(y_true, y_pred)), 2),
            'MAE': round(mean_absolute_error(y_true, y_pred), 2),
            'R2 Score': round(r2_score(y_true, y_pred), 4)
        })
    return pd.DataFrame(results)

# --- 5. Comparaison Train vs Test ---

metrics_train = get_metrics(modele, X_train_tensor, y_train, scaler_y, "Training")
metrics_test = get_metrics(modele, X_test_tensor, y_test, scaler_y, "Test")

final_report = pd.concat([metrics_train, metrics_test], ignore_index=True).sort_values(by='Cible')
print("\n--- Rapport de Performance Optimisé ---")
print(final_report.to_string(index=False))


--- Rapport de Performance Optimisé ---
 Dataset      Cible   RMSE    MAE  R2 Score
Training Carat (ct)   0.05   0.03    0.9904
    Test Carat (ct)   0.05   0.03    0.9911
Training  Price ($) 771.63 398.60    0.9626
    Test  Price ($) 753.06 395.31    0.9643


p=0.2
modele = nn.Sequential(
    nn.Linear(input_dimension,32),
    nn.ReLU(),
    nn.Dropout(p),
    nn.Linear(32,64),
    nn.ReLU(),
    nn.Linear(64,16),
    nn.ReLU(),
    nn.Linear(16,output_dimension),
)

criterion = (nn.MSELoss())
optimizer = optim.Adam(modele.parameters(), lr=0.01)
summary(modele, input_size=input_dimension, output_size=output_dimension)
100 epoch

--- Rapport de Performance (Train vs Test) ---
 Dataset      Cible   RMSE    MAE  R2 Score
Training Carat (ct)   0.04   0.02    0.9927
    Test Carat (ct)   0.04   0.02    0.9946
Training  Price ($) 743.12 394.60    0.9653
    Test  Price ($) 707.80 386.80    0.9685



p=0.2
modele = nn.Sequential(
    nn.Linear(input_dimension,16),
    nn.ReLU(),
    nn.Dropout(p),
    nn.Linear(16,8),
    nn.ReLU(),
    nn.Linear(8,output_dimension),
)

criterion = (nn.MSELoss())
optimizer = optim.Adam(modele.parameters(), lr=0.01)
summary(modele, input_size=input_dimension, output_size=output_dimension)
epoch 100
--- Rapport de Performance (Train vs Test) ---
 Dataset      Cible   RMSE    MAE  R2 Score
Training Carat (ct)   0.07   0.04    0.9798
    Test Carat (ct)   0.06   0.04    0.9859
Training  Price ($) 826.07 486.83    0.9571
    Test  Price ($) 787.17 480.52    0.9610